In [50]:
#!/usr/bin/env python
from pathlib import Path
import torch

from src.vae_module.vae_module import VAEModule
from src.data.components.mp_dataset import MPDataset
from src.data.schema import CrystalBatch

f1 = '/home/u5bd/lleon.u5bd/chemeleon2/logs/train_vae/runs/2025-09-26_21-29-51/checkpoints'
f1 = '/home/u5bd/lleon.u5bd/chemeleon2/logs/train_vae/runs/2025-09-26_21-33-21/checkpoints'
f1 = '/home/u5bd/lleon.u5bd/chemeleon2/logs/train_vae/runs/2025-09-26_21-33-22/checkpoints'

CKPT_DIR = Path(
    f1 
)
CKPT_NAME = "last.ckpt"  # change to epoch_1949.ckpt etc. if you prefer
DATA_ROOT = Path("/home/u5bd/lleon.u5bd/chemeleon/data/mp-40")


In [51]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ckpt_path = CKPT_DIR / CKPT_NAME

model = VAEModule.load_from_checkpoint(str(ckpt_path), map_location=device)
model = model.to(device)
model.eval()

dataset = MPDataset(root=str(DATA_ROOT), split="train")
first_graph = dataset[0]
batch = CrystalBatch.collate([first_graph]).to(device)

print(model.encoder.index_embedding_type)


/home/u5bd/lleon.u5bd/miniforge3/envs/chem/lib/python3.12/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/u5bd/lleon.u5bd/miniforge3/envs/chem/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:210: Attribute 'encoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['encoder'])`.
/home/u5bd/lleon.u5bd/miniforge3/envs/chem/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:210: Attribute 'decoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['decoder'])`.


none


In [52]:
with torch.no_grad():
    encoded = model.encode(batch)
    z  = encoded["posterior"].mode()
    perm01 = torch.arange(z.size(0), device=z.device); perm01[[0, 1]] = perm01[[1, 0]]
    perm23 = torch.arange(z.size(0), device=z.device); perm23[[2, 3]] = perm23[[3, 2]]
    z1, z2 = z[perm01], z[perm23]
    decoded_z  = model.decode({**encoded, "x": z,  "z": z})
    decoded_z1 = model.decode({**encoded, "x": z1, "z": z1})
    decoded_z2 = model.decode({**encoded, "x": z2, "z": z2})

    reconstructed  = model.reconstruct(decoded_z, batch)
    reconstructed1 = model.reconstruct(decoded_z1, batch)
    reconstructed2 = model.reconstruct(decoded_z2, batch)


material_id = dataset.df.loc[0, "material_id"]
print(f"Material ID: {material_id}")
print("Latent sample shape:", z.shape)
print("Decoded lengths (scaled):", decoded_z["lengths"][0])
print("Decoded frac coords shape:", decoded_z["frac_coords"].shape)
structure = reconstructed.to_structure()[0]
structure1 = reconstructed1.to_structure()[0]
structure2 = reconstructed2.to_structure()[0]
print("Reconstructed structure formula:", structure.composition.formula)


Material ID: mp-1105194
Latent sample shape: torch.Size([19, 8])
Decoded lengths (scaled): tensor([2.1459, 2.5487, 2.7819])
Decoded frac coords shape: torch.Size([19, 3])
Reconstructed structure formula: Ho5 Mo2 O12


In [53]:
pos = reconstructed.pos
pos1 = reconstructed1.pos
pos2 = reconstructed2.pos

inv01 = torch.empty_like(perm01); inv01[perm01] = torch.arange(perm01.size(0), device=perm01.device)
inv23 = torch.empty_like(perm23); inv23[perm23] = torch.arange(perm23.size(0), device=perm23.device)

pos1_unperm = pos1[inv01]
pos2_unperm = pos2[inv23]


In [54]:
print(model.encoder.index_embedding_type)
(
    torch.mean(pos1_unperm - pos), torch.mean(pos2_unperm - pos),
    torch.max(torch.abs(pos1_unperm - pos)), torch.max(torch.abs(pos2_unperm - pos)),
)

none


(tensor(3.0129e-08),
 tensor(1.3888e-07),
 tensor(1.4752e-06),
 tensor(1.9073e-06))